In [29]:
import pandas as pd
import re

In [30]:
#Import the Emploitic scrape snapshot
df = pd.read_csv('job_listings.csv')
df.head(20)

,job title,hiring organization,date posted,valid through,sector,job location,experience required,education requirement,type of contract,available posts,description,url
0,Office Manager,Arab Leasing Corporation,2025-01-12T15:44:30.306Z,2025-03-13T23:59:59.000Z,"Banque, Assurance, Finance","Alger, Algérie",3 À 5 Ans,"Master 2, Ingéniorat, Bac + 5",CDI,1 poste ouvert,"<p style=""text-align: center; "">&nbsp;Office M...",https://emploitic.com/entreprises/arab-leasing...
1,Spécialiste Ressources Humaines,Arab Leasing Corporation,2025-01-12T15:08:34.612Z,2025-03-13T23:59:59.000Z,"Banque, Assurance, Finance","Alger, Algérie",3 À 5 Ans,"Licence (LMD), Bac + 3",CDI,1 poste ouvert,<br />Missions du poste :<ul><li>Administratio...,https://emploitic.com/entreprises/arab-leasing...
2,Chargé des Opérations,GoDyma,2025-01-12T12:24:33.136Z,2025-03-13T23:59:59.000Z,Services,"Oran, Algérie | Oran, Daïra Oran, Oran, Algérie",1 À 2 Ans,NaN,CDD Ou Mission,4 postes ouverts,&nbsp; Annonce de Recrutement<br /> <br class=...,https://emploitic.com/entreprises/godyma/offre...
3,"Technicien Engins spécialisés en Diesel, Hydra...",SARL MANIMUS,2025-01-12T10:27:13.000Z,2025-02-03T23:59:59.000Z,Services,"Alger, Algérie",3 À 5 Ans,"Master 1, Licence Bac + 4 | Formation Profess...",CDD Ou Mission,4 postes ouverts,MISSIONS : - Effectuer la maintenance préventi...,https://emploitic.com/entreprises/sarl-manimus...
4,Juriste,Importante entreprise,2025-01-12T00:00:03.608Z,2025-02-10T23:59:59.000Z,Services,"Alger, Algérie",3 À 5 Ans,"Licence (LMD), Bac + 3 | Master 2, Ingéniorat,...",NaN,1 poste ouvert,"<ul><li> Grande Société, acteur majeur du sect...",https://emploitic.com/offres-d-emploi/services...
5,Responsable Réseau de Distribution Commerciale,Soficlef,2025-01-11T22:13:22.282Z,2025-03-12T23:59:59.000Z,Industries,"Si Mustapha, Daïra Isser, Boumerdès, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 1, Licence Ba...",CDI,1 poste ouvert,"<b style=""font-size: 1rem;"">Présentation de l'...",https://emploitic.com/entreprises/soficlef/off...
6,CADRE FINANCIER,Entreprise anonyme,2025-01-11T12:55:37.766Z,2025-03-12T23:59:59.000Z,"Distribution, Commerce","Oued Smar, Daïra El Harrach, Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 1, Licence Ba...",CDD Ou Mission,1 poste ouvert,"<b style=""font-size: 1rem;"">Missions principal...",https://emploitic.com/offres-d-emploi/distribu...
7,CADRE COMPTABLE,Entreprise anonyme,2025-01-11T12:51:07.038Z,2025-03-12T23:59:59.000Z,"Distribution, Commerce","Oued Smar, Daïra El Harrach, Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 1, Licence Ba...",CDD Ou Mission,1 poste ouvert,"<b style=""font-size: 1rem;"">Missions et Activi...",https://emploitic.com/offres-d-emploi/distribu...
8,Comptable,Entreprise anonyme,2025-01-11T12:49:11.450Z,2025-03-12T23:59:59.000Z,"Distribution, Commerce","Oued Smar, Daïra El Harrach, Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 1, Licence Ba...",CDD Ou Mission,2 postes ouverts,"<b style=""font-size: 1rem;"">Missions et activi...",https://emploitic.com/offres-d-emploi/distribu...
9,Ingénieur en informatique Senior,Entreprise anonyme,2025-01-11T12:45:47.921Z,2025-03-12T23:59:59.000Z,"Distribution, Commerce","Oued Smar, Daïra El Harrach, Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Master 1, Licence Bac + 4 | Master 2, Ingénio...",CDD Ou Mission,1 poste ouvert,"<b style=""font-size: 1rem;"">Missions principal...",https://emploitic.com/offres-d-emploi/distribu...


In [31]:
#Drop the none Functioning Links
df.dropna(subset=['job title'], inplace=True)

In [32]:
#Reset the index
df = df.reset_index(drop=True)

In [33]:
def clean_html_text(html):
    # Replace <b>, <u> and other tags
    html = re.sub(r'<b>|</b>|<u>|</u>', '', html)
    html = re.sub(r'<.*?>', '', html)
    
    # Replace <p> with a newline and strip extra spaces
    html = re.sub(r'<p>', '\n', html)
    html = re.sub(';', '//', html)
    html = re.sub('&nbsp', '//', html)
    html = re.sub(r'</p>', '', html)
    
    # Replace bullet points (·) with a dash or another symbol
    html = html.replace('·', '')
    html = html.replace(':', '')
    html = html.replace(',', '')
    html = html.replace('.', '')
    html = html.replace('-', '')
    html = html.replace('/', '')
    
    # Remove extra spaces
    html = re.sub(r'\s+', ' ', html)
    
    # Clean up single quotes
    html = html.replace('’', "'")
    
    # Trim the text (leading and trailing spaces)
    html = html.strip()
    
    return html

In [34]:
#Further cleaning of the description
df['description'] = df['description'].apply(clean_html_text)

In [35]:
def extract_edu(description):
    bac_pattern = r"\b(?:\w*BAC\s?\+?\s?\d{1,2})\b"
    diploma_pattern = r"(\w*diplôme|Diplômé|Ingéniorat|ingenieur|technicien|BTS|Master|formation|Bachelor's|University degree|University|Formation|Master's|niveau universitaire)\s+([a-zA-Zéàûêï']+\s*){1,5}"
    
    # Search for BAC qualification
    bac_match = re.search(bac_pattern, description, re.IGNORECASE)
    if bac_match:
        return bac_match.group(0)  # If BAC is found, return it
        # Search for diploma qualification
    diploma_match = re.search(diploma_pattern, description, re.IGNORECASE)
    if diploma_match:
        return diploma_match.group(0)  # If diploma is found, return it
    return pd.NA  # Return None if neither BAC nor diploma is found

In [36]:
nan_edu = df[df['education requirement'].isna()]
nan_edu

,job title,hiring organization,date posted,valid through,sector,job location,experience required,education requirement,type of contract,available posts,description,url
2,Chargé des Opérations,GoDyma,2025-01-12T12:24:33.136Z,2025-03-13T23:59:59.000Z,Services,"Oran, Algérie | Oran, Daïra Oran, Oran, Algérie",1 À 2 Ans,NaN,CDD Ou Mission,4 postes ouverts,Annonce de Recrutement SociétéSARL GODYMA Expr...,https://emploitic.com/entreprises/godyma/offre...
17,Agent Technique / Technicien Helpdesk IT,Institution Etatique,2025-01-10T11:32:28.000Z,2025-02-16T23:59:59.000Z,"Fonction Publique, Administration","Alger, Algérie",1 À 2 Ans | 3 À 5 Ans | 6 À 10 Ans,NaN,CDI,1 poste ouvert,Institution étatique recrute un Agent Techniqu...,https://emploitic.com/offres-d-emploi/fonction...
19,Responsable administrative,Digital I.A,2025-01-09T18:16:04.000Z,2025-03-09T23:59:59.000Z,Services,"Alger, Algérie",Moins D’un An | 1 À 2 Ans,NaN,NaN,1 poste ouvert,Description de poste Nous sommes à la recherch...,https://emploitic.com/entreprises/digital-ia/o...
27,Key Account Manager,Iris Studio,2025-01-09T16:23:52.630Z,2025-03-10T23:59:59.000Z,Services,"Constantine, Algérie | Oran, Algérie | Alger, ...",3 À 5 Ans,NaN,CDD Ou Mission,3 postes ouverts,À propos de nous Agence de communication leade...,https://emploitic.com/entreprises/iris-studio/...
31,Contrôleur de Gestion et de recouvrement,Importante entreprise,2025-01-09T15:24:13.555Z,2025-03-10T23:59:59.000Z,Services,"Ouled Fayet, Daïra Cheraga, Alger, Algérie | A...",1 À 2 Ans,NaN,CDD Ou Mission,1 poste ouvert,Secteur d'activité Services dans le domaine de...,https://emploitic.com/offres-d-emploi/services...
...,...,...,...,...,...,...,...,...,...,...,...,...
2739,Chargé Importation,Euro Protection Algérie,2024-11-13T15:19:48.762Z,2025-01-12T23:59:59.000Z,"Distribution, Commerce","Boumerdès, Algérie",3 À 5 Ans,NaN,NaN,1 poste ouvert,Missions du poste Veille sur le bon déroulemen...,https://emploitic.com/entreprises/euro-protect...
2741,Electromécanicien / Electronicien,Importante entreprise,2024-11-13T15:13:07.098Z,2025-01-12T23:59:59.000Z,"Distribution, Commerce","Alger, Algérie",3 À 5 Ans,NaN,NaN,1 poste ouvert,Description du poste L'Electromécanicien Elect...,https://emploitic.com/offres-d-emploi/distribu...
2742,Technicien SAV (engins roulants),Importante entreprise,2024-11-13T15:11:35.556Z,2025-01-12T23:59:59.000Z,"Distribution, Commerce","Alger, Algérie",3 À 5 Ans,NaN,NaN,1 poste ouvert,Description du poste Le technicien engins roul...,https://emploitic.com/offres-d-emploi/distribu...
2762,Généraliste RH,Iris Studio,2024-11-13T10:38:21.226Z,2025-01-12T23:59:59.000Z,Services,"Oran, Daïra Oran, Oran, Algérie",1 À 2 Ans | 3 À 5 Ans | 6 À 10 Ans | Plus de 1...,NaN,CDD Ou Mission,1 poste ouvert,Offre d'emploi Généraliste en Ressources Humai...,https://emploitic.com/entreprises/iris-studio/...


In [37]:
#Imputate Education requirments from the description
for i in nan_edu.index:
    str = extract_edu(df.loc[i, 'description'])
    print(str)
    df.loc[i, 'education requirement'] = str
df

Bac +1
Bac +2
formation du personnel sur les procédures 
Bac+45
<NA>
BTS en administration Droit gestion des 
formation et le développement ainsi que 
Bac+3
<NA>
Formation BTS en mécanique maintenance industrielle 
Bac + 3
BAC+5
Master enmécanique industrielle électronique etc
<NA>
Diplôme d'études supérieures en Finance Gestion 
Formation d'infographie Maitrise des langues arabe 
<NA>
<NA>
Formation universitaire en contr
Formation Niveau universitaire 
Diplôme Licence en Marketing ou dipl
<NA>
Formation complémentaire souhaitable Techniques de vente 
Bac + 4
Bac + 4
<NA>
FORMATION DISPENSEE AINSI QUE LE STAGE 
formation et à la mise en 
Formation et développement Concevoir et mettre 
formation continue Effectuer des audits réguliers 
formation pour sensibiliser les employés à 
formation figurant dans les différentes brochures 
diplôme d'études supérieures en Chimie pharmaceutique 
diplôme d'études supérieures en Chimie pharmaceutique 
<NA>
Diplôme en commerce marketing ou domaine 
Di

,job title,hiring organization,date posted,valid through,sector,job location,experience required,education requirement,type of contract,available posts,description,url
0,Office Manager,Arab Leasing Corporation,2025-01-12T15:44:30.306Z,2025-03-13T23:59:59.000Z,"Banque, Assurance, Finance","Alger, Algérie",3 À 5 Ans,"Master 2, Ingéniorat, Bac + 5",CDI,1 poste ouvert,Office Manager Missions du poste Structurer et...,https://emploitic.com/entreprises/arab-leasing...
1,Spécialiste Ressources Humaines,Arab Leasing Corporation,2025-01-12T15:08:34.612Z,2025-03-13T23:59:59.000Z,"Banque, Assurance, Finance","Alger, Algérie",3 À 5 Ans,"Licence (LMD), Bac + 3",CDI,1 poste ouvert,Missions du poste Administration RHAssurer les...,https://emploitic.com/entreprises/arab-leasing...
2,Chargé des Opérations,GoDyma,2025-01-12T12:24:33.136Z,2025-03-13T23:59:59.000Z,Services,"Oran, Algérie | Oran, Daïra Oran, Oran, Algérie",1 À 2 Ans,Bac +1,CDD Ou Mission,4 postes ouverts,Annonce de Recrutement SociétéSARL GODYMA Expr...,https://emploitic.com/entreprises/godyma/offre...
3,"Technicien Engins spécialisés en Diesel, Hydra...",SARL MANIMUS,2025-01-12T10:27:13.000Z,2025-02-03T23:59:59.000Z,Services,"Alger, Algérie",3 À 5 Ans,"Master 1, Licence Bac + 4 | Formation Profess...",CDD Ou Mission,4 postes ouverts,MISSIONS Effectuer la maintenance préventive d...,https://emploitic.com/entreprises/sarl-manimus...
4,Juriste,Importante entreprise,2025-01-12T00:00:03.608Z,2025-02-10T23:59:59.000Z,Services,"Alger, Algérie",3 À 5 Ans,"Licence (LMD), Bac + 3 | Master 2, Ingéniorat,...",NaN,1 poste ouvert,Grande Société acteur majeur du secteur MARITI...,https://emploitic.com/offres-d-emploi/services...
...,...,...,...,...,...,...,...,...,...,...,...,...
2771,Cadre chargé de la Sous Traitance,Entreprise anonyme,2024-11-13T08:49:26.541Z,2025-01-12T23:59:59.000Z,"Energie, Mines, Matière Première","Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 1, Licence Ba...",CDD Ou Mission,2 postes ouverts,Profil recherché Formation (titres et diplômes...,https://emploitic.com/offres-d-emploi/energie-...
2772,Ingénieur Junior Process,Entreprise anonyme,2024-11-13T08:47:36.138Z,2025-01-12T23:59:59.000Z,"Energie, Mines, Matière Première","Alger, Algérie",Sans Expérience | Moins D’un An | 1 À 2 Ans | ...,"Master 2, Ingéniorat, Bac + 5",CDD Ou Mission,2 postes ouverts,Exigences particulières Dégagé du service nati...,https://emploitic.com/offres-d-emploi/energie-...
2773,Responsable Trade Marketing,EL GHAZOU EURL,2024-11-13T08:20:59.989Z,2025-01-12T23:59:59.000Z,Industries,"Dar El Beïda, Daïra Dar El Beïda, Alger, Algérie",3 À 5 Ans | 6 À 10 Ans,"Licence (LMD), Bac + 3 | Master 2, Ingéniorat,...",CDD Ou Mission,1 poste ouvert,El Ghazou est une entreprise dynamique et inno...,https://emploitic.com/entreprises/el-ghazou-eu...
2774,comptable,Imetal,2024-11-13T07:36:58.910Z,2025-01-12T23:59:59.000Z,Services,"Alger, Algérie",Moins D’un An,Formation Professionnelle,CDD Ou Mission,1 poste ouvert,Missions Tenue de la comptabilité généraleÉlab...,https://emploitic.com/entreprises/imetal/offre...


In [38]:
education_dict = {
    "Bac + 7": ["Magistère Bac + 7", "Bac + 7", "Doctorat", "7"],
    "Bac + 5": ["Master 2", "Ingéniorat", "Bac + 5", "Bac+45", "Bac+5", "Master", "master", "MASTER", "5", "Diplômé", "Master's", "University degree", "Bachelor's",'BAC +5','RecherchéBac + 45','BAC +05','Bac +35','BAC +45','Bac + 05','BAC + 5', 'Bac +5','SkillsBac +5', 'BAC+5', "University", "niveau universitaire","Diplôme d'état", "Diplôme Master"],
    "Bac + 4": ["Master 1", "Licence Bac + 4", "Bac + 4", "Bac+45", "4", "Diplômé", "Ingéniorat", "Master", "Bachelor's",'BAC +45'],
    "Bac + 3": ["Licence (LMD)", "Bac + 3", "bac+3", "Bac+3", "BAC+3", "3",'Bac +23', "Diplômé", "Bachelor's","Diplôme Licence",'Bac+23','Bac +35','Bac + 03','BAC + 3'],
    "Bac + 2": ["TS Bac +2", "Bac +2", "BTS", "2",'Bac +23',"Bac+2",'Bac+23'],
    "Bac + 1": ["Bac +1", "1"],
    "Bac": ["Baccalauréat"],
    "Formation Professionnelle": ["Formation Professionnelle", "formation", "Formation", "technicien", "Formation Pro"],
    "Certification": ["Certification", "certifié", "Certifié", "Certifications", "Certification professionnelle"],
    "Niveau Terminal": ["Niveau Terminal", "Terminal Level", "Niveau secondaire", "Terminal level"],
    "Niveau Secondaire": ["Niveau Secondaire", "High School", "Secondary Level", "Non Diplômante"],
    "Diplôme": ["Diplôme d'études supérieures", "Diplôme universitaire","Diplôme en informatique", "Diplôme en gestion", "Diplôme en commerce","recherchéDiplôme"],
    "Niveau universitaire": ["Université", "Université sans diplôme", "Diplôme universitaire","Niveau universitaire"],
}

In [39]:
# Corrected function to standardize each requirement level
def standardize_education(level):
    edu_req = []
    if pd.notna(level):  # Check if the value is not NaN
        # Split the string into parts based on "|" and ","
        components = [part.strip() for sub in level.split("|") for part in sub.split(",")]
        print(components)  # Debugging step to check the split components

        # Loop through each part and check for a match in the education_dict
        for part in components:
            for key, values in education_dict.items():
                if part in values:
                    print(f"Matched part: {part}")  # Debugging step to confirm match
                    edu_req.append(key)  # Return the first matching key (standardized value)
        if edu_req != []:
            return list(set(edu_req))
        # If no match found, return the original string itself
        else:
            return level
    else:
        return level  # Return the original value if it's NaN

In [40]:
# Apply the standardization function to the 'education requirement' column
df['education requirement'] = df['education requirement'].apply(standardize_education)

['Master 2', 'Ingéniorat', 'Bac + 5']
Matched part: Master 2
Matched part: Ingéniorat
Matched part: Ingéniorat
Matched part: Bac + 5
['Licence (LMD)', 'Bac + 3']
Matched part: Licence (LMD)
Matched part: Bac + 3
['Bac +1']
Matched part: Bac +1
['Master 1', 'Licence  Bac + 4', 'Formation Professionnelle']
Matched part: Master 1
Matched part: Formation Professionnelle
['Licence (LMD)', 'Bac + 3', 'Master 2', 'Ingéniorat', 'Bac + 5']
Matched part: Licence (LMD)
Matched part: Bac + 3
Matched part: Master 2
Matched part: Ingéniorat
Matched part: Ingéniorat
Matched part: Bac + 5
['Licence (LMD)', 'Bac + 3', 'Master 1', 'Licence  Bac + 4', 'Master 2', 'Ingéniorat', 'Bac + 5']
Matched part: Licence (LMD)
Matched part: Bac + 3
Matched part: Master 1
Matched part: Master 2
Matched part: Ingéniorat
Matched part: Ingéniorat
Matched part: Bac + 5
['Licence (LMD)', 'Bac + 3', 'Master 1', 'Licence  Bac + 4', 'Master 2', 'Ingéniorat', 'Bac + 5']
Matched part: Licence (LMD)
Matched part: Bac + 3
Matche

In [41]:
def standardize_education2(level):
    edu_req = []
    if not isinstance(level, list):  
        print(f'level {level} type: {type(level)}')
        if pd.notna(level) and level.strip():  
            bac_pattern = r"\bBAC\s?\+?\s?\d{1,3}\b|Bac\s*\+\s*\d{1,3}|Bac\+\d{1,3}"
            bac_match = re.search(bac_pattern, level, re.IGNORECASE)
            if bac_match:
                bac = re.search(r'\b(?:Bac\s*\+\s*(\d+)|BAC\s*\+\s*(\d+)|Bac\+\s*(\d+))\b', bac_match.group(), re.IGNORECASE)  # Extract number
                if bac:
                    for key, values in education_dict.items():
                        # Check if the bac number matches any item in the values
                        if any(bac.group() in value for value in values):
                            print(f'matched part: {bac.group()}')
                            edu_req.append(key)
            else:
                diploma_pattern = r"\b(formation|diplôme|Diplômé|Ingéniorat|ingenieur|technicien|Technicien|Technicien|BTS|Master|Bachelor's|University degree|University|Master's|niveau universitaire|Niveau universitaire|Certification|Niveau Secondaire|Niveau Terminal)\b"
                diploma = re.search(diploma_pattern, level, re.IGNORECASE)
                
                if diploma:
                    for key, values in education_dict.items():
                        # Check if the diploma match exists in the values
                        if any(diploma.group() in value for value in values):
                            print(f'matched part: {diploma.group()}')
                            edu_req.append(key)
            if edu_req:  # Return the unique list if matches found
                return list(set(edu_req))
            else:
                return level  # Return original level if no match found
        else:
            return level  # Return original value if NaN or empty
    else:
        return level  # Return the original value if level is a list

In [42]:
# Apply the second standardization function to the 'education requirement' column
df['education requirement'] = df['education requirement'].apply(standardize_education2)

level formation du personnel sur les procédures  type: <class 'str'>
matched part: formation
level <NA> type: <class 'pandas._libs.missing.NAType'>
level BTS en administration Droit gestion des  type: <class 'str'>
matched part: BTS
level formation et le développement ainsi que  type: <class 'str'>
matched part: formation
level <NA> type: <class 'pandas._libs.missing.NAType'>
level Formation BTS en mécanique maintenance industrielle  type: <class 'str'>
matched part: Formation
level Master enmécanique industrielle électronique etc type: <class 'str'>
matched part: Master
matched part: Master
level <NA> type: <class 'pandas._libs.missing.NAType'>
level Diplôme d'études supérieures en Finance Gestion  type: <class 'str'>
matched part: Diplôme
matched part: Diplôme
matched part: Diplôme
matched part: Diplôme
level Formation d'infographie Maitrise des langues arabe  type: <class 'str'>
matched part: Formation
level <NA> type: <class 'pandas._libs.missing.NAType'>
level <NA> type: <class 'p

In [43]:
def finalize_edu_imputation(level):
    if not isinstance(level, list):
        return 'None spécifiée'
    else:
        return level 

In [44]:
#Mark none imputated rows as none specified
df['education requirement'] = df['education requirement'].apply(finalize_edu_imputation)

In [45]:
def extract_type_contract(description):
    cdd_pattern = r"\bCDD\b"
    cdi_pattern = r"\bCDI\b"
    cdd_match = re.search(cdd_pattern, description, re.IGNORECASE)
    if cdd_match:
        return "CDD Ou Mission"  
    cdi_match = re.search(cdi_pattern, description, re.IGNORECASE)
    if cdi_match:
        return "CDI"  
    return 'None spécifiée' 

In [46]:
# Apply the function and assign back only to rows with NaN
# Imputate the contract type from the description and assign back only to rows with NaN
df.loc[pd.isna(df['type of contract']), 'type of contract'] = (
    df.loc[pd.isna(df['type of contract']), 'description']
    .apply(extract_type_contract)
)


In [47]:
#Standrdize datetime columns
df['date posted'] = pd.to_datetime(df['date posted']).dt.tz_localize(None)
df['valid through'] = pd.to_datetime(df['valid through']).dt.tz_localize(None)

In [48]:
def extract_post_nb(level):
    posts_nb_pattern = r'^\d{1,2}'
    posts_nb_match = re.search(posts_nb_pattern, level, re.IGNORECASE)
    if posts_nb_match:
        return int(posts_nb_match.group(0))
    else:
        return level

In [49]:
#Standrdize the number of available posts 
df['available posts'] = df['available posts'].apply(extract_post_nb)

In [50]:
def standardize_experience(level):
    if pd.isna(level):
        return []  # Handle missing values
    experience_list = [exp.strip() for exp in level.split("|")]  # Strip extra spaces
    return experience_list

In [51]:
#Standrdize expeience requirments
df['experience required'] = df['experience required'].apply(standardize_experience)

In [52]:
wilayas_dict = {
    "Adrar": "Adrar",
    "Chlef": "Chlef",
    "Laghouat": "Laghouat",
    "Oum El Bouaghi": "Oum El Bouaghi",
    "Batna": "Batna",
    "Béjaïa": "Béjaïa",
    "Biskra": "Biskra",
    "Béchar": "Béchar",
    "Blida": "Blida",
    "Bouira": "Bouira",
    "Tamanrasset": "Tamanrasset",
    "Tébessa": "Tébessa",
    "Tlemcen": "Tlemcen",
    "Tiaret": "Tiaret",
    "Tizi Ouzou": "Tizi Ouzou",
    "Alger": "Alger",
    "Djelfa": "Djelfa",
    "Jijel": "Jijel",
    "Sétif": "Sétif",
    "Saïda": "Saïda",
    "Skikda": "Skikda",
    "Sidi Bel Abbès": "Sidi Bel Abbès",
    "Annaba": "Annaba",
    "Guelma": "Guelma",
    "Constantine": "Constantine",
    "Médéa": "Médéa",
    "Mostaganem": "Mostaganem",
    "M'Sila": "M'Sila",
    "Mascara": "Mascara",
    "Ouargla": "Ouargla",
    "Oran": "Oran",
    "El Bayadh": "El Bayadh",
    "Illizi": "Illizi",
    "Bordj Bou Arréridj": "Bordj Bou Arréridj",
    "Boumerdès": "Boumerdès",
    "El Tarf": "El Tarf",
    "Tindouf": "Tindouf",
    "Tissemsilt": "Tissemsilt",
    "El Oued": "El Oued",
    "Khenchela": "Khenchela",
    "Souk Ahras": "Souk Ahras",
    "Tipaza": "Tipaza",
    "Mila": "Mila",
    "Aïn Defla": "Aïn Defla",
    "Naâma": "Naâma",
    "Aïn Témouchent": "Aïn Témouchent",
    "Ghardaïa": "Ghardaïa",
    "Relizane": "Relizane"
}

In [53]:
def standardize_location(level):
    location = [] 
    if pd.isna(level):
        return []  

    location_list = [loc.strip() for loc in level.split("|")]
    
    for part in location_list:
        for key, value in wilayas_dict.items():
            if value.lower() in part.lower():  # Case-insensitive match
                print(f"Matched part: {part}") 
                location.append(key)  
    
    return list(set(location)) if location else level

In [54]:
#Standrdize locations
df['job location'] = df['job location'].apply(standardize_location)

Matched part: Alger, Algérie
Matched part: Alger, Algérie
Matched part: Oran, Algérie
Matched part: Oran, Daïra Oran, Oran, Algérie
Matched part: Alger, Algérie
Matched part: Alger, Algérie
Matched part: Si Mustapha, Daïra Isser, Boumerdès, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Cheraga, Daïra Cheraga, Alger, Algérie
Matched part: Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Oued Smar, Daïra El Harrach, Alger, Algérie
Matched part: Alger, Algérie
Matched part: Alger, Algérie
Matched part: Alger, Algérie
Matched part: Alger, Algérie
Matched part: Alger, Algérie
Ma

In [55]:
df.drop(columns=['description', 'url'], inplace=True)

In [56]:
df.to_csv('job_listings_cleaned.csv', index=False)